In [ ]:
import numpy as np
import pandas
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from PIL import Image
from scipy import ndimage
from skimage import measure

In [ ]:
dataf = pandas.read_csv(r'letter-recognition-new.csv')

In [ ]:
dataf.info()

In [ ]:
dataf.describe()

In [ ]:
y = dataf.pop('letter')

In [ ]:
def norm(df):
    norm_df = df.copy()
    columns_to_normalize = df.columns

    for column in columns_to_normalize:
        min_val = norm_df[column].min()
        max_val = norm_df[column].max()
    
        norm_df[column] = (norm_df[column] - min_val) / (max_val - min_val)
    return norm_df

In [ ]:
dataf_norm = norm(dataf)

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(dataf_norm.corr(), annot=True, cmap="coolwarm", fmt='.2f')
plt.show

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(dataf_norm,y,test_size=0.2)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
#looking for optimal k
best_k = 1
for k in range(2, 21):
    test_knn = KNeighborsClassifier(k)
    best_knn = KNeighborsClassifier(best_k)
    
    test_knn.fit(x_train, y_train)
    best_knn.fit(x_train, y_train)
    
    if test_knn.score(x_test, y_test) > best_knn.score(x_test, y_test):
        best_k = k
        
print(best_k)

In [ ]:
results = []

for i in range(1,21):
    knnTest = KNeighborsClassifier(i)
    knnTest.fit(x_train, y_train)
    result = knnTest.score(x_test, y_test)
    results.append(result)

In [ ]:
arguments = [i for i in range(1,21)]
plt.figure(figsize=(10, 5))
plt.plot(arguments, results, label = 'Dokładność', color = 'blue')
plt.title('Zależność dokładności modelu do ilości sąsiadów dla algorytmu kNN')
plt.xlabel('Ilość sąsiadów k')
plt.ylabel('Dokładność')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


knn_lib=KNeighborsClassifier(best_k)
knn_lib.fit(x_train, y_train)
knn_lib.score(x_test, y_test)

In [ ]:
args = []
accs = []
for i in range(100):
    args.append(i)
    xt, xte, yt, yte = train_test_split(dataf_norm,y,test_size=0.2)
    knn = KNeighborsClassifier(6)
    knn.fit(xt, yt)
    accs.append(knn.score(xte, yte))

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(args, accs, label = 'Dokładność dla danej próby', color = 'blue')
plt.axhline(np.mean(accs), label = "Średnia dokładność", color = 'red', linestyle = 'dashed')
plt.title('Wartości dokładności dla 100 prób')
plt.xlabel('Numer próby')
plt.ylabel('Dokładność')
plt.legend()
plt.grid(True)
plt.show()